### for region specific visualzations 
Environment: region_specfic
find all ids belonging to a parent (ie. 315, for isocortex), create mask and visualize using napari

generate average image and multiply with the mask!


In [41]:
import os

import pandas as pd

import numpy as np

import Neuron_analysis as na

import skimage
from skimage import io

import SimpleITK as sitk

import re
from tqdm import tqdm

import napari

import nrrd

In [44]:
def isolate_region(this_region):
    
    '''''' 
    
    child_ids= atlas_labels.loc[atlas_labels['structure_id_path'].str.contains(f'/{this_region}/', case=False)].id
    # child_ids

    # find all regions with regionX inside the structure id path
    # for example 315 - isocortex, 985- primary motor region
    
    this_mask= np.zeros(annot_h.shape)
    # create empty array to store the mask
    for i in tqdm(child_ids.values):
        this_mask[annot_h==i]=1
        #loop through all the region id and assign value of 1
    # empty_array now is a matrix of 1 and 0s
    
    return this_mask

def isolate_region_2(regions_list):
    
    '''useful for isolating one layer of multiple region, ie MOs 2/3 and MOp 2/3''' 
    
    #child_ids= atlas_labels.loc[atlas_labels['structure_id_path'].str.contains('|'.join(regions_list), case=False)].id
    # child_ids

    # find all regions with regionX inside the structure id path
    # for example 315 - isocortex, 985- primary motor region
    
    this_mask= np.zeros(annot_h.shape)
    # create empty array to store the mask
    for i in tqdm(regions_list):
        this_mask[annot_h==i]=1
        #loop through all the region id and assign value of 1
    # empty_array now is a matrix of 1 and 0s
    
    return this_mask

def save_tiff(numpystack, outname):
    
    '''save numpy stack as tiff stack and also save sum projections in horizontal, coronal, and saggital views'''
    
    print('Starting to saving tif files..')
    
    #io.imsave(outname+'_axons.tif',numpystack)
    
    sum_0=np.sum(numpystack, axis=0)
    sum_1=np.sum(numpystack, axis=1)
    sum_2=np.sum(numpystack, axis=2)
    io.imsave(outname+ '_sum_0.tif',sum_0)
    #io.imsave(outname+ '_sum_1.tif',sum_1)
    #io.imsave(outname+ '_sum_2.tif',sum_2)

def save_tiff_mask(numpystack, outname):

    ''' most useful for saving max projection for region masks, to be used as outlines in illustrators
    save numpy stack as png max projections in horizontal, coronal, and saggital views '''

    print('Starting to saving tif files..')

    #io.imsave(outname+'_axons.tif',numpystack)

    sum_0=np.max(numpystack, axis=0)
    #sum_1=np.sum(numpystack, axis=1)
    #sum_2=np.sum(numpystack, axis=2)
    
    io.imsave(outname+ '_max_0.tif',sum_0)
    #io.imsave(outname+ '_max_1.tif',sum_1)
    #io.imsave(outname+ '_max_2.tif',sum_2)

In [3]:
file_name = r'C:/Users/liu/mouse_connectivity/annotation/ccf_2017/annotation_25.nrrd'
data_array, metadata = nrrd.read(file_name)

annot_h=np.moveaxis(data_array, 1, 0)
print('Converted to horizontal atlas with shape', annot_h.shape)
# reslice corontal atlas to horizontal atlas that is consistent with our sample's orientation

atlas_labels=pd.read_csv('D:\Allenbrainatlas\ARA_25_micron_mhd_ccf2017\labels.csv')
#read labels

Converted to horizontal atlas with shape (320, 528, 456)


In [8]:
#viewer = napari.Viewer()

#viewer.add_image(template, name='template')
#viewer.add_labels(annot_h, name='atlas')


In [4]:
#axons1= sitk.ReadImage(r'D:\viral_results\s2\AL278_axons.tif')
#axons1=sitk.GetArrayFromImage(axons2) 
# issue with image with 64 bit samples, since the normalized images have values like 10e-5
# hence we work with npy files directly

region='s1'
mouseline='ras'

file_path= f'D:\\viral_results\\s1_s2\\averages\\{region}_{mouseline}_avg.npy'

axons=np.load(file_path)
axons[0,0,0]=0
axons[0,1,0]=0
# dont forget to change the [0,0,0] position from the total number of axons back to 0!
#viewer.add_image(axons)


Stuff to try

isocortec 315

MOp1, MOs1 [320,656]

MOp 2/3 943,  MOs 2/3 962. [943,962]

MOp5, MOs 5. [648,767]

MOp6a, MOs 6a [844,1021]

MOp6b, MOs 6b [882,1085]


region_dict= {
    '1':[320,656],
    '23':[943,962],
    '5':[648,767],
    '6a':[844,1021],
    '6b':[882,1085]
}

In [6]:
region_id= ['315']
# can be a list such as region_id= ['943', '962'] , but will have two separate files
# or can be one big region ie. 315 the isocortex wich have multiple child - use isolate_region

for i in region_id: 
    
    #region_mask= isolate_region(i)
    axons_masked= axons*region_mask
    
    out_name= f'D:\\viral_results\\s1_s2\\region\\{i}_{region}_{mouseline}_avg'
    
    save_tiff(axons_masked,out_name)

100%|████████████████████████████████████████████████████████████████████████████████| 374/374 [00:48<00:00,  7.69it/s]


Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_4124\2350469427.py:34: UserWarning: D:\viral_results\s1_s2\region\315_s2_ras_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_4124\2350469427.py:35: UserWarning: D:\viral_results\s1_s2\region\315_s2_ras_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_4124\2350469427.py:36: UserWarning: D:\viral_results\s1_s2\region\315_s2_ras_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


In [5]:
# loop sections
regions=['s1', 's2']
mouse_lines=['ras','scn','tlx','sim1','rbp','ntsr']

In [22]:
region_dict= {
    '1':[320,656],
    '23':[943,962],
    '5':[648,767],
    '6a':[844,1021],
    '6b':[882,1085]
}

In [21]:
region_id= [320,656]
layer= '1'

# cannot be string to use isolate region2
region_mask= isolate_region_2(region_id)
# generate mask outside of the loop since it will be the same!

#region_id= ['315']
#region_mask= isolate_region(region_id)
# generate mask outside of the loop since it will be the same!

for i in regions:
    for j in mouse_lines:
        file_path= f'D:\\viral_results\\s1_s2\\averages\\{i}_{j}_avg.npy'

        axons=np.load(file_path)
        axons[0,0,0]=0
        axons[0,1,0]=0
        
        axons_masked= axons*region_mask
        
        out_name= f'D:\\viral_results\\s1_s2\\region\\MO\\{layer}_{i}_{j}_avg'
        save_tiff(axons_masked,out_name)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.95it/s]


Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_15532\280341463.py:50: UserWarning: D:\viral_results\s1_s2\region\MO\1_s1_ras_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)


Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_15532\280341463.py:50: UserWarning: D:\viral_results\s1_s2\region\MO\1_s1_scn_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)


...

C:\Users\liu\AppData\Local\Temp\ipykernel_15532\280341463.py:50: UserWarning: D:\viral_results\s1_s2\region\MO\1_s2_sim1_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)


Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_15532\280341463.py:50: UserWarning: D:\viral_results\s1_s2\region\MO\1_s2_rbp_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)


Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_15532\280341463.py:50: UserWarning: D:\viral_results\s1_s2\region\MO\1_s2_ntsr_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)


In [4]:
### plot region mask itself for outlines

In [45]:
region_dict= {
    '1':[320,656],
    '23':[943,962],
    '5':[648,767],
    '6a':[844,1021],
    '6b':[882,1085]
}

In [46]:
for key,value in region_dict.items(): 

    region_mask= isolate_region_2(value)
    # generate mask outside of the loop since it will be the same!

    out_name= f'D:\\viral_results\\s1_s2\\region\\MO\\{key}_mask'
    save_tiff_mask(region_mask,out_name)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.17it/s]


Starting to saving tif files..


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.10it/s]


Starting to saving tif files..


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.42it/s]


Starting to saving tif files..


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.17it/s]


Starting to saving tif files..


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.40it/s]


Starting to saving tif files..


### section to create averaged image for a given injection site for a given mouse line

6 mouselines *2 injection sites =12 total

In [2]:
def save_tiff(numpystack, outname):
    
    '''save numpy stack as tiff'''
    
    print('Starting to saving tif files..')
    
    io.imsave(outname+'_axons.tif',numpystack)
    
    sum_0=np.sum(numpystack, axis=0)
    sum_1=np.sum(numpystack, axis=1)
    sum_2=np.sum(numpystack, axis=2)
    io.imsave(outname+ '_sum_0.tif',sum_0)
    io.imsave(outname+ '_sum_1.tif',sum_1)
    io.imsave(outname+ '_sum_2.tif',sum_2)

#### s1 sim1 samples

In [29]:
# s1 sim1 samples

name_list= ['AL207','AL209','AL273']

s1_sim1= np.zeros((320, 528, 456, len(name_list)))
# create big empty array to store the axons arrays, with the first 3 dimensions as horizontal allen atlas and last dimension as number of samples in this group

In [31]:
for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\s1\\{i}.npy')
    s1_sim1[:,:,:,count]=axon

loading AL207.npy
loading AL209.npy
loading AL273.npy


In [44]:
s1_sim1_avg= np.average(s1_sim1, axis=3)
s1_sim1_avg.shape

s1_sim1_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save('D:\\viral_results\\s1_s2\\s1_sim1_avg.npy', s1_sim1_avg)
# save this averaged array


In [48]:
# verify axons in plotly and then save as tiff stack & sum projections

s1_sim1_avg[0,0,0]=0
# dont forget to change the [0,0,0] position from the total number of axons back to 0!

s1_sim1_avg[0,1,0]=0
# dont forget to change the [0,1,0] position from the total number of samples in this group back to 0!

viewer.add_image(s1_sim1_avg)

save_tiff(s1_sim1_avg, 'D:\\viral_results\\s1_s2\\s1_sim1_avg')

Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_12116\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s1_sim1_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)
C:\Users\liu\AppData\Local\Temp\ipykernel_12116\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s1_sim1_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_12116\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s1_sim1_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_12116\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s1_sim1_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s2 sim1 samples

In [5]:
region='s2'
mouse_line= 'sim1'

name_list= ['AL281','AL286','AL321','AL322']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save('D:\\viral_results\\s1_s2\\s2_sim1_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL281.npy
loading AL286.npy
loading AL321.npy
loading AL322.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s2_sim1_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s2_sim1_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s2_sim1_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s2_sim1_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s1 rbp samples

In [7]:
region='s1'
mouse_line= 'rbp'

name_list= ['AL211','AL215','AL318']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL211.npy
loading AL215.npy
loading AL318.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s1_rbp data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s2 rbp samples

In [7]:
region='s2'
mouse_line= 'rbp'

name_list= ['AL288','AL326','AL327']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL211.npy
loading AL215.npy
loading AL318.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s1_rbp data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s1_rbp_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s1 tlx samples

In [10]:
region='s1'
mouse_line= 'tlx'

name_list= ['AL213','AL313','AL314']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL213.npy
loading AL313.npy
loading AL314.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s1_tlx_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s1_tlx data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s1_tlx_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s1_tlx_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s1_tlx_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s2 tlx samples

In [11]:
region='s2'
mouse_line= 'tlx'

name_list= ['AL278','AL280','AL319']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL278.npy
loading AL280.npy
loading AL319.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s2_tlx_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s2_tlx data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s2_tlx_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s2_tlx_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s2_tlx_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s1 ras samples

In [12]:
region='s1'
mouse_line= 'ras'

name_list= ['AL254','AL255','AL257']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL254.npy
loading AL255.npy
loading AL257.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s1_ras_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s1_ras data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s1_ras_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s1_ras_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s1_ras_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s2 ras samples 

In [4]:
region='s2'
mouse_line= 'ras'

name_list= ['AL303','AL332','AL333']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL303.npy
loading AL332.npy
loading AL333.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_7772\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s2_ras_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s2_ras data


C:\Users\liu\AppData\Local\Temp\ipykernel_7772\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s2_ras_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_7772\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s2_ras_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_7772\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s2_ras_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s1 scn sample

In [14]:
region='s1'
mouse_line= 'scn'

name_list= ['AL290','AL291','AL292','AL293']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL290.npy
loading AL291.npy
loading AL292.npy
loading AL293.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s1_scn_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s1_scn data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s1_scn_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s1_scn_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s1_scn_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s2 scn sample

In [16]:
region='s2'
mouse_line= 'scn'

name_list= ['AL290','AL292','AL323']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL290.npy
loading AL292.npy
loading AL323.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s2_scn_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s2_scn data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s2_scn_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s2_scn_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s2_scn_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s1 ntsr samples

In [17]:
region='s1'
mouse_line= 'ntsr'

name_list= ['AL274','AL285','AL311']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL274.npy
loading AL285.npy
loading AL311.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s1_ntsr_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s1_ntsr data


C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s1_ntsr_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s1_ntsr_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_17252\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s1_ntsr_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


#### s2 ntsr samples

In [3]:
region='s2'
mouse_line= 'ntsr'

name_list= ['AL274','AL310','AL330']

emptyarray= np.zeros((320, 528, 456, len(name_list)))

for count, i in enumerate (name_list):
    print (f'loading {i}.npy')
    axon= np.load (f'D:\\viral_results\\{region}\\{i}.npy')
    emptyarray[:,:,:,count]=axon
    
emptyarray_avg= np.average(emptyarray, axis=3)
emptyarray_avg.shape

emptyarray_avg[0,1,0]=len(name_list)
# store the number of samples used to calculate this average at the [0,1,0] location
# so that the value at [0,0,0] multiply by value at [0,1,0] equals the total number of axons for all samples in this group

np.save(f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg.npy', emptyarray_avg)
# save this averaged array

emptyarray_avg[0,0,0]=0
emptyarray_avg[0,1,0]=0

save_tiff(emptyarray_avg, f'D:\\viral_results\\s1_s2\\{region}_{mouse_line}_avg')

print(f'Done saving {region}_{mouse_line} data')

loading AL274.npy
loading AL310.npy
loading AL330.npy
Starting to saving tif files..


C:\Users\liu\AppData\Local\Temp\ipykernel_6316\828751201.py:7: UserWarning: D:\viral_results\s1_s2\s2_ntsr_avg_axons.tif is a low contrast image
  io.imsave(outname+'_axons.tif',numpystack)


Done saving s2_ntsr data


C:\Users\liu\AppData\Local\Temp\ipykernel_6316\828751201.py:12: UserWarning: D:\viral_results\s1_s2\s2_ntsr_avg_sum_0.tif is a low contrast image
  io.imsave(outname+ '_sum_0.tif',sum_0)
C:\Users\liu\AppData\Local\Temp\ipykernel_6316\828751201.py:13: UserWarning: D:\viral_results\s1_s2\s2_ntsr_avg_sum_1.tif is a low contrast image
  io.imsave(outname+ '_sum_1.tif',sum_1)
C:\Users\liu\AppData\Local\Temp\ipykernel_6316\828751201.py:14: UserWarning: D:\viral_results\s1_s2\s2_ntsr_avg_sum_2.tif is a low contrast image
  io.imsave(outname+ '_sum_2.tif',sum_2)


### section holder